# Environment setup

In [ ]:
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(patchwork)
library(Seurat)
library(SeuratDisk)
library(ggplot2)
library(dplyr)
library(DESeq2)
library(pheatmap)

In [ ]:
%%R
cluster = TRUE

In [ ]:
%%R
# Check if the directory exists before setting it
if (cluster) {
  dir_path <- "/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_microglia/GSE171266"
} else {
  dir_path <- "/home/michal/WSL_GitHub/SRF_microglia/data/GSE171266"
}

if (dir.exists(dir_path)) {
  setwd(dir_path)
} else {
  warning(paste("Directory", dir_path, "does not exist. Working directory not changed."))
}

# Load single-cell data
load_10x_data <- function(path, sample_name) {
  if (!dir.exists(path)) {
    stop(paste("Directory", path, "does not exist."))
  }
  data <- Read10X(data.dir = path)
  seurat_obj <- CreateSeuratObject(counts = data, project = sample_name, min.cells = 3, min.features = 200)
  seurat_obj$sample <- sample_name
  return(seurat_obj)
}

# Load raw data

In [ ]:
%%R
wt_data <- load_10x_data("./data/GSM5221533_1", "WT_D686D")
mut_data <- load_10x_data("./data/GSM5221534_2", "Mut_D868N")

In [ ]:
%%R
# Merge datasets
combined_data <- merge(wt_data, mut_data)

## Save raw data

In [ ]:
%%R
# Display summary of combined_data
print(combined_data)

# Save the Seurat object to a file
saveRDS(combined_data, file = "./output/combined_data.rds")

# Confirm the file was saved
if (file.exists("./output/combined_data.rds")) {
  cat("Seurat object saved successfully to combined_data.rds\n")
} else {
  cat("Error: Failed to save Seurat object\n")
}

In [ ]:
%%R
# Load the Seurat object from the saved file
combined_data <- readRDS("./output/combined_data.rds")

# Verify that the object was loaded correctly
if (is(combined_data, "Seurat")) {
  cat("Seurat object loaded successfully from combined_data.rds\n")
  print(combined_data)
} else {
  cat("Error: Failed to load Seurat object or loaded object is not a Seurat object\n")
}

# Fltere and process data

In [ ]:
%%R
# Calculate quality control metrics
combined_data[["percent.mt"]] <- PercentageFeatureSet(combined_data, pattern = "^MT-")

In [ ]:
%%R
head(combined_data@meta.data)

In [ ]:
%%R
library(cowplot)
plot_qc_metrics <- function(seurat_obj) {
  # Ensure we're using the correct assay
  DefaultAssay(seurat_obj) <- "RNA"
  
  # Extract metadata and ensure 'sample' is a factor
  metadata <- seurat_obj@meta.data %>%
    mutate(sample = factor(sample))
  
  # Custom theme
  my_theme <- theme_cowplot() +
    theme(
      axis.title = element_text(size = 12, face = "bold"),
      axis.text = element_text(size = 10),
      plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
      legend.position = "bottom",
      legend.title = element_blank(),
      strip.background = element_blank(),
      strip.text = element_text(size = 12, face = "bold")
    )

  # Color palette
  colors <- c("#E69F00", "#56B4E9")

  # Function to create violin plot with reduced point density
  create_violin <- function(feature, title, yintercept = NULL) {
    p <- ggplot(metadata, aes(x = sample, y = !!sym(feature), fill = sample)) +
      geom_violin(alpha = 0.7) +
      geom_jitter(size = 0.1, alpha = 0.1, width = 0.2) +
      scale_fill_manual(values = colors) +
      labs(title = title, y = title, x = "") +
      my_theme

    if (!is.null(yintercept)) {
      p <- p + geom_hline(yintercept = yintercept, color = "red", linetype = "dashed")
    }
    
    return(p)
  }

  p1 <- create_violin("nFeature_RNA", "Number of Genes", c(200, 3000)) +
    scale_y_continuous(labels = scales::comma)

  p2 <- create_violin("nCount_RNA", "Number of UMIs", 10000) +
    scale_y_continuous(labels = scales::comma)

  p3 <- create_violin("percent.mt", "Percentage of Mitochondrial Genes", 20)

  # Scatter plot with hexbin to reduce overplotting
  p4 <- ggplot(metadata, aes(x = nCount_RNA, y = nFeature_RNA, color = sample)) +
    geom_hex(bins = 100, aes(fill = ..count..), show.legend = FALSE) +
    scale_fill_viridis_c() +
    geom_vline(xintercept = 10000, color = "red", linetype = "dashed") +
    geom_hline(yintercept = c(200, 3000), color = "red", linetype = "dashed") +
    labs(title = "Genes vs UMIs", x = "Number of UMIs", y = "Number of Genes") +
    scale_color_manual(values = colors) +
    my_theme +
    scale_x_continuous(labels = scales::comma) +
    scale_y_continuous(labels = scales::comma)

  # Combine plots
  combined_plot <- (p1 + p2) / (p3 + p4) +
    plot_layout(guides = "collect") &
    theme(legend.position = "bottom")

  return(combined_plot)
}

# Create and display plots in notebook
print(plot_qc_metrics(combined_data))

In [ ]:
%%R
# Filter cells (you can modify these thresholds based on the plots)
combined_data <- subset(combined_data, subset = nFeature_RNA > 200 & nFeature_RNA < 3000 & 
                        nCount_RNA < 10000 & percent.mt < 20)

In [ ]:
%%R
# Create and display plots in notebook
print(plot_qc_metrics(combined_data))

In [ ]:
%%R
# Normalize data
combined_data <- NormalizeData(combined_data)

In [ ]:
%%R
# Find variable features
combined_data <- FindVariableFeatures(combined_data, selection.method = "vst", nfeatures = 2000)

In [ ]:
%%R
# Scale data
# all_genes <- rownames(combined_data)
variable_genes <- VariableFeatures(combined_data)
combined_data <- ScaleData(combined_data, features = variable_genes)

In [ ]:
%%R
# Run PCA
combined_data <- RunPCA(combined_data, features = VariableFeatures(object = combined_data))

In [ ]:
%%R
# Find neighbors and run UMAP
combined_data <- FindNeighbors(combined_data, dims = 1:40)
combined_data <- RunUMAP(combined_data, dims = 1:40)

In [ ]:
%%R
# Save the Seurat object before clustering
saveRDS(combined_data, file = "./output/GSE171266_before_clustering.rds")

# Load the saved Seurat object

In [ ]:
%%R
# Load the saved Seurat object
combined_data <- readRDS("./output/GSE171266_before_clustering.rds")

In [ ]:
%%R
combined_data

In [ ]:
%%R
# Perform clustering
# Set random seed for reproducibility
set.seed(42)

# Increase the memory limit for R
options(future.globals.maxSize = 64 * 1024^3)  # Set to 8GB

# Set future.seed to TRUE to ensure proper random number generation
future::plan("multicore", workers = parallel::detectCores() - 1)

# Use Louvain algorithm with optimized parameters
combined_data <- FindClusters(
  object = combined_data,
  resolution = 1.0,
  algorithm = 1,
  method = "igraph",  # Use igraph method for larger datasets
  n.start = 20,       # Increase number of random starts for better optimization
  n.iter = 20,        # Increase max iterations for more thorough optimization
  random.seed = 42,   # Set random seed for reproducibility
  verbose = TRUE      # Print progress
)

# Print clustering progress
print(paste("Number of clusters:", length(unique(Idents(combined_data)))))

# Calculate and print modularity score
modularity_score <- modularity(
  graph = combined_data@graphs$RNA_snn, 
  membership = Idents(combined_data)
)
print(paste("Modularity score:", modularity_score))

# If memory issues persist, try these alternatives:
# 1. Subset cells
# subset_cells <- sample(colnames(combined_data), size = 10000)
# combined_data <- subset(combined_data, cells = subset_cells)

# 2. Use a subset of features
# combined_data <- FindClusters(combined_data, features = VariableFeatures(combined_data)[1:1000], resolution = 1.0, algorithm = 1)

# 3. Disable future and run sequentially
# combined_data <- FindClusters(combined_data, resolution = 1.0, algorithm = 1, future.use.globals = FALSE)

IOStream.flush timed out
IOStream.flush timed out


In [ ]:
%%R
# Save the Seurat object
saveRDS(combined_data, file = "./output/GSE171266_processed.rds")

In [ ]:
%%R
# Load the saved Seurat object
combined_data <- readRDS("./output/GSE171266_processed.rds")

In [ ]:
%%R
# Plot UMAP
p1 <- DimPlot(combined_data, reduction = "umap", group.by = "sample")
p2 <- DimPlot(combined_data, reduction = "umap", group.by = "seurat_clusters")
print(p1 + p2)